In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [3]:
import chardet
file_path = 'Superstore.csv'
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
    return result['encoding']

detected_encoding = detect_encoding(file_path)
print(f"Detected encoding: {detected_encoding}")


Detected encoding: Windows-1252


In [4]:
ss = pd.read_csv('Superstore.csv', encoding=detected_encoding)
ss.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,13-06-2013,17-06-2013,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [5]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

# Pricing strategy

Sales by Category

In [6]:
ss.groupby('Category')['Sales'].sum().sort_values(ascending=False)

Category
Technology         836154.0330
Furniture          741999.7953
Office Supplies    719047.0320
Name: Sales, dtype: float64

Technology has the most sales among all categories.

In [7]:
ss[ss['Category'] == 'Technology'].groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False)

Sub-Category
Phones         330007.054
Machines       189238.631
Accessories    167380.318
Copiers        149528.030
Name: Sales, dtype: float64

Phones has the most sales in technology category.

In [8]:
ss[ss['Sub-Category'] == 'Phones'].groupby('Product Name')['Sales'].sum().sort_values(ascending=False)

Product Name
Samsung Galaxy Mega 6.3                                                13943.668
Apple iPhone 5                                                         12996.600
Wilson Electronics DB Pro Signal Booster                                8878.400
Mitel MiVoice 5330e IP Phone                                            7699.720
Samsung Galaxy S III - 16GB - pebble blue (T-Mobile)                    7139.796
                                                                         ...    
Shocksock Galaxy S4 Armband                                               63.510
Anker 24W Portable Micro USB Car Charger                                  61.544
SKILCRAFT Telephone Shoulder Rest, 2" x 6.5" x 2.5", Black                56.164
Sannysis Cute Owl Design Soft Skin Case Cover for Samsung Galaxy S4       13.860
PNY Rapid USB Car Charger - Black                                          9.588
Name: Sales, Length: 189, dtype: float64

Samsung Galaxy Mega 6.3 and Apple iphone 5 have the most sales in phone sub-category.

Profit by category

In [9]:
ss.groupby('Category')['Profit'].sum().sort_values(ascending=False)

Category
Technology         145454.9481
Office Supplies    122490.8008
Furniture           18451.2728
Name: Profit, dtype: float64

In [18]:
ss[ss['Category'] == 'Technology']['Profit'].std()/ss[ss['Category'] == 'Office Supplies']['Profit'].std()

2.6006653372152964

In [19]:
import scipy.stats as stats
data1 = ss[ss['Category'] == 'Technology']['Profit']
data2 = ss[ss['Category'] == 'Office Supplies']['Profit']
t_statistic, p_value = stats.ttest_ind(data1, data2)

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Check if the p-value is significant (using a common significance level of 0.05)
if p_value < 0.05:
    print("The means are significantly different.")
else:
    print("The means are not significantly different.")

t-statistic: 8.687539082544111
p-value: 4.4542772738946324e-18
The means are significantly different.


In [22]:
print('tech avg : {}'.format(data1.mean()))
print('office supplies avg : {}'.format(data2.mean()))

tech avg : 78.75200221981586
office supplies avg : 20.327049585131117


Technology has the most profit among all categories.

In [11]:
ss[ss['Category'] == 'Technology'].groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False)

Sub-Category
Phones         330007.054
Machines       189238.631
Accessories    167380.318
Copiers        149528.030
Name: Sales, dtype: float64

In [23]:
ss[ss['Sub-Category'] == 'Phones'].groupby('Product Name')['Profit'].sum().sort_values(ascending=False)

Product Name
Samsung Galaxy Mega 6.3                                                                                 1822.7566
Apple iPhone 5                                                                                          1611.5784
Panasonic KX-TG9471B                                                                                    1168.1004
Panasonic KX-TG9541B DECT 6.0 Digital 2-Line Expandable Cordless Phone With Digital Answering System    1147.4235
Samsung Galaxy S4 Mini                                                                                  1104.4765
                                                                                                          ...    
Cisco IP Phone 7961G-GE VoIP phone                                                                      -119.1190
Samsung Galaxy Note 2                                                                                   -128.7972
LG Exalt                                                                   

If we have a pricing strategy, we should focus on upselling in the phone-technology category, particularly for the Samsung Galaxy Mega 6.3 and the Apple iPhone 5